# DATA EXTRACTION FROM MIMIC-III FOR THE AI-CLINICIAN

This notebook contains SQL queries required to recreate dataset for the publication: https://www.nature.com/articles/s41591-018-0213-5


Recreating the dataset is a 3-step process:
- Data extraction done here in SQL, sub-tables exported as csv files
- Data imported in Matlab for:
    - Sepsis-3 cohort definition (suspected infection + SOFA score)
    - Construction of the final dataset
    
It uses some queries from the MIMIC-III repo: https://github.com/MIT-LCP/mimic-code

Dr Matthieu Komorowski, Imperial College London 2019.

If you use any of this code, please reference appropriately.

This code is distributed in the hope that it will be useful, but WITHOUT ANY WARRANTY, without even the implied warranty of MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
import os
import pandas as pd
from google.cloud import bigquery

# --- CONFIGURATION ---
# Your Billing Project ID (from your previous error logs)
# You must use YOUR project to pay for the query computation
BILLING_PROJECT_ID = "mimic-analysis-480222"

# --- INITIALIZE CLIENT ---
# We do NOT use os.environ["GOOGLE_APPLICATION_CREDENTIALS"]
# The client will automatically find the login from Step 1.
try:
    client = bigquery.Client(project=BILLING_PROJECT_ID)
    print(f"Successfully connected using billing project: {BILLING_PROJECT_ID}")
except Exception as e:
    print(f"Authentication failed. Did you run 'gcloud auth application-default login'?\nError: {e}")
    exit()

# --- THE QUERY ---
# Accessing the table shown in your screenshot: physionet-data.mimiciii_clinical
query = """
SELECT subject_id, gender, dob
FROM `physionet-data.mimiciii_clinical.patients`
LIMIT 10
"""

print("Running query...")

try:
    # Run the query
    query_job = client.query(query)
    
    # Convert to DataFrame
    df = query_job.to_dataframe()
    
    print("Query Successful! Here is the data:")
    print(df.head())

except Exception as e:
    print("An error occurred during the query:")
    print(e)

/rds/general/user/js3320/home/anaconda3/envs/EH_new/lib/python3.10/site-packages/google/api_core/_python_version_support.py:266: FutureWarning: You are using a Python version (3.10.16) which Google will stop supporting in new releases of google.api_core once it reaches its end of life (2026-10-04). Please upgrade to the latest Python version, or at least Python 3.11, to continue receiving updates for google.api_core past that date.
  warnings.warn(message, FutureWarning)
/rds/general/user/js3320/home/anaconda3/envs/EH_new/lib/python3.10/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Successfully connected using billing project: mimic-analysis-480222
Running query...
Query Successful! Here is the data:
   subject_id gender        dob
0          56      F 1804-01-02
1       18848      F 2042-08-21
2       61056      F 2067-04-11
3       26889      F 2115-11-04
4       18333      F 2094-01-21


# EXTRACTION OF SUB-TABLES

## 'culture' items

These correspond to blood/urine/CSF/sputum cultures etc.

In [8]:
import pandas as pd
from google.cloud import bigquery
import os

# --- SETUP ---
project_id = 'mimic-analysis-480222' 
client = bigquery.Client(project=project_id)
exportdir = 'data_extraction/'
if not os.path.exists(exportdir):
    os.makedirs(exportdir)

# --- QUERY ---
query = """
SELECT 
    subject_id, 
    hadm_id, 
    icustay_id,  
    UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
    itemid
FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE itemid IN (
    6035,3333,938,941,942,4855,6043,2929,225401,225437,225444,225451,225454,
    225814,225816,225817,225818,225722,225723,225724,225725,225726,225727,
    225728,225729,225730,225731,225732,225733,227726,70006,70011,70012,
    70013,70014,70016,70024,70037,70041,225734,225735,225736,225768,70055,
    70057,70060,70063,70075,70083,226131,80220
)
ORDER BY subject_id, hadm_id, charttime
"""

print("Running Culture query...")
query_job = client.query(query)
d = query_job.to_dataframe()
d.to_csv(exportdir + 'culture.csv', index=False, sep='|')
print(f"Success! Saved culture.csv ({len(d)} rows).")

/rds/general/user/js3320/home/anaconda3/envs/EH_new/lib/python3.10/site-packages/google/auth/_default.py:108: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


Running Culture query...
Success! Saved culture.csv (19343 rows).


## Microbiologyevents

Extracts date & time of all microbiologyevents (regardless of whether they are positive or negative).

In [9]:
print("Extracting Microbiology Data...")
query_micro = """
SELECT 
    subject_id, 
    hadm_id, 
    UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
    UNIX_SECONDS(TIMESTAMP(chartdate)) as chartdate 
FROM `physionet-data.mimiciii_clinical.microbiologyevents`
"""

df_micro = client.query(query_micro).to_dataframe()
df_micro.to_csv(exportdir + 'microbio.csv', index=False, sep='|')
print(f"Saved microbio.csv ({len(df_micro)} rows)")

Extracting Microbiology Data...
Saved microbio.csv (631726 rows)


## Antibiotics administration

In [10]:
print("Extracting Antibiotics Data...")

gsn_codes = (
    "'002542','002543','007371','008873','008877','008879','008880','008935','008941',"
    "'008942','008943','008944','008983','008984','008990','008991','008992','008995',"
    "'008996','008998','009043','009046','009065','009066','009136','009137','009162',"
    "'009164','009165','009171','009182','009189','009213','009214','009218','009219',"
    "'009221','009226','009227','009235','009242','009263','009273','009284','009298',"
    "'009299','009310','009322','009323','009326','009327','009339','009346','009351',"
    "'009354','009362','009394','009395','009396','009509','009510','009511','009544',"
    "'009585','009591','009592','009630','013023','013645','013723','013724','013725',"
    "'014182','014500','015979','016368','016373','016408','016931','016932','016949',"
    "'018636','018637','018766','019283','021187','021205','021735','021871','023372',"
    "'023989','024095','024194','024668','025080','026721','027252','027465','027470',"
    "'029325','029927','029928','037042','039551','039806','040819','041798','043350',"
    "'043879','044143','045131','045132','046771','047797','048077','048262','048266',"
    "'048292','049835','050442','050443','051932','052050','060365','066295','067471'"
)

query_abx = f"""
SELECT 
    hadm_id, 
    icustay_id, 
    UNIX_SECONDS(TIMESTAMP(startdate)) as startdate, 
    UNIX_SECONDS(TIMESTAMP(enddate)) as enddate
FROM `physionet-data.mimiciii_clinical.prescriptions`
WHERE gsn IN ({gsn_codes})
ORDER BY hadm_id, icustay_id
"""

df_abx = client.query(query_abx).to_dataframe()
df_abx.to_csv(exportdir + 'abx.csv', index=False, sep='|')
print(f"Saved abx.csv ({len(df_abx)} rows)")

Extracting Antibiotics Data...
Saved abx.csv (84491 rows)


## Demographics

Requires building **public table** *Elixhauser_Quan* first: https://github.com/MIT-LCP/mimic-code/blob/master/concepts/comorbidity/elixhauser-quan.sql

In [11]:
print("Extracting Demographics Data...")

query_demog = """
SELECT 
    ad.subject_id, 
    ad.hadm_id, 
    i.icustay_id, 
    UNIX_SECONDS(TIMESTAMP(ad.admittime)) as admittime, 
    UNIX_SECONDS(TIMESTAMP(ad.dischtime)) as dischtime,
    ROW_NUMBER() OVER (PARTITION BY ad.subject_id ORDER BY i.intime ASC) as adm_order,
    CASE 
        WHEN i.first_careunit='NICU' THEN 5 
        WHEN i.first_careunit='SICU' THEN 2 
        WHEN i.first_careunit='CSRU' THEN 4 
        WHEN i.first_careunit='CCU' THEN 6 
        WHEN i.first_careunit='MICU' THEN 1 
        WHEN i.first_careunit='TSICU' THEN 3 
    END as unit,
    UNIX_SECONDS(TIMESTAMP(i.intime)) as intime, 
    UNIX_SECONDS(TIMESTAMP(i.outtime)) as outtime, 
    i.los,
    TIMESTAMP_DIFF(TIMESTAMP(i.intime), TIMESTAMP(p.dob), SECOND)/86400 as age,
    UNIX_SECONDS(TIMESTAMP(p.dob)) as dob, 
    UNIX_SECONDS(TIMESTAMP(p.dod)) as dod,
    p.expire_flag,
    CASE WHEN p.gender='M' THEN 1 WHEN p.gender='F' THEN 2 END as gender,
    CAST(TIMESTAMP_DIFF(TIMESTAMP(p.dod), TIMESTAMP(ad.dischtime), SECOND) <= 24*3600 AS INT64) as morta_hosp,
    CAST(TIMESTAMP_DIFF(TIMESTAMP(p.dod), TIMESTAMP(i.intime), SECOND) <= 90*24*3600 AS INT64) as morta_90,
    congestive_heart_failure + cardiac_arrhythmias + valvular_disease + pulmonary_circulation + 
    peripheral_vascular + hypertension + paralysis + other_neurological + chronic_pulmonary + 
    diabetes_uncomplicated + diabetes_complicated + hypothyroidism + renal_failure + liver_disease + 
    peptic_ulcer + aids + lymphoma + metastatic_cancer + solid_tumor + rheumatoid_arthritis + 
    coagulopathy + obesity + weight_loss + fluid_electrolyte + blood_loss_anemia + 
    deficiency_anemias + alcohol_abuse + drug_abuse + psychoses + depression as elixhauser
FROM `physionet-data.mimiciii_clinical.admissions` ad
JOIN `physionet-data.mimiciii_clinical.icustays` i ON ad.hadm_id = i.hadm_id
JOIN `physionet-data.mimiciii_clinical.patients` p ON p.subject_id = i.subject_id
LEFT JOIN `physionet-data.mimiciii_derived.elixhauser_quan` elix ON elix.hadm_id = ad.hadm_id
ORDER BY subject_id ASC, intime ASC
"""

try:
    df_demog = client.query(query_demog).to_dataframe()
    df_demog.to_csv(exportdir + 'demog.csv', index=False, sep='|')
    print(f"Saved demog.csv ({len(df_demog)} rows)")
except Exception as e:
    print(f"Error: {e}")

Extracting Demographics Data...
Saved demog.csv (61532 rows)


## Vitals from Chartevents

Divided into 10 chunks for speed. Each chunk is around 170 MB.

In [12]:
print("--- Extracting Vitals (Chunks) ---")

# Item IDs for vitals
vitals_items = "(467, 470,471,223834,227287,194,224691,226707,226730,581,580,224639,226512,198,228096,211,220045,220179,225309,6701,6,227243,224167,51,455, 220181,220052,225312,224322,6702,443,52,456,8368,8441,225310,8555,8440,220210,3337,224422,618,3603,615,220277,646,834,3655,223762,223761,678,220074,113,492,491,8448,116,1372,1366,228368,228177,626,223835,3420,160,727,190,220339,506,505,224700,224686,224684,684,224421,224687,450,448,445,224697,444,224695,535,224696,543,3083,2566,654,3050,681,2311)"

for i in range(0, 100000, 10000):
    print(f"Processing chunk starting at {i}...")
    
    query_vitals = f"""
    SELECT DISTINCT 
        icustay_id, 
        UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
        itemid, 
        CASE 
            WHEN value = 'None' THEN '0' 
            WHEN value = 'Ventilator' THEN '1' 
            WHEN value = 'Cannula' THEN '2' 
            WHEN value = 'Nasal Cannula' THEN '2' 
            WHEN value = 'Face Tent' THEN '3' 
            WHEN value = 'Aerosol-Cool' THEN '4' 
            WHEN value = 'Trach Mask' THEN '5' 
            WHEN value = 'Hi Flow Neb' THEN '6' 
            WHEN value = 'Non-Rebreather' THEN '7' 
            WHEN value = '' THEN '8'  
            WHEN value = 'Venti Mask' THEN '9' 
            WHEN value = 'Medium Conc Mask' THEN '10' 
            ELSE CAST(valuenum AS STRING) 
        END as valuenum 
    FROM `physionet-data.mimiciii_clinical.chartevents` 
    WHERE icustay_id >= {200000+i} AND icustay_id < {210000+i} 
    AND value IS NOT NULL 
    AND itemid IN {vitals_items}
    ORDER BY icustay_id, charttime
    """
    
    d = client.query(query_vitals).to_dataframe()
    d.to_csv(exportdir + 'ce' + str(i) + str(i+10000) + '.csv', index=False, sep='|')
    print(f"Saved chunk {i}")

--- Extracting Vitals (Chunks) ---
Processing chunk starting at 0...
Saved chunk 0
Processing chunk starting at 10000...
Saved chunk 10000
Processing chunk starting at 20000...
Saved chunk 20000
Processing chunk starting at 30000...
Saved chunk 30000
Processing chunk starting at 40000...
Saved chunk 40000
Processing chunk starting at 50000...
Saved chunk 50000
Processing chunk starting at 60000...
Saved chunk 60000
Processing chunk starting at 70000...
Saved chunk 70000
Processing chunk starting at 80000...
Saved chunk 80000
Processing chunk starting at 90000...
Saved chunk 90000


## Labs from Chartevents

In [13]:
print("--- Extracting Labs (Chartevents) ---")

lab_items = "(829,1535,227442,227464,4195,3726,3792,837,220645,4194,3725,3803,226534,1536,4195,3726,788,220602,1523,4193,3724,226536,3747,225664,807,811,1529,220621,226537,3744,781,1162,225624,3737,791,1525,220615,3750,821,1532,220635,786,225625,1522,3746,816,225667,3766,777,787,770,3801,769,3802,1538,848,225690,803,1527,225651,3807,1539,849,772,1521,227456,3727,227429,851,227444,814,220228,813,220545,3761,226540,4197,3799,1127,1542,220546,4200,3834,828,227457,3789,825,1533,227466,3796,824,1286,1671,1520,768,220507,815,1530,227467,780,1126,3839,4753,779,490,3785,3838,3837,778,3784,3836,3835,776,224828,3736,4196,3740,74,225668,1531,227443,1817,228640,823,227686)"

query_labs_ce = f"""
SELECT 
    icustay_id, 
    UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
    itemid, 
    valuenum
FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE valuenum IS NOT NULL 
AND icustay_id IS NOT NULL 
AND itemid IN {lab_items}
ORDER BY icustay_id, charttime, itemid
"""

d = client.query(query_labs_ce).to_dataframe()
d.to_csv(exportdir + 'labs_ce.csv', index=False, sep='|')
print(f"Saved labs_ce.csv ({len(d)} rows)")

--- Extracting Labs (Chartevents) ---
Saved labs_ce.csv (10895296 rows)


## Labs from Labevents

In [14]:
print("--- Extracting Labs (Labevents) ---")

lab_items_le = "(50971,50822,50824,50806,50931,51081,50885,51003,51222,50810,51301,50983,50902,50809,51006,50912,50960,50893,50808,50804,50878,50861,51464,50883,50976,50862,51002,50889,50811,51221,51279,51300,51265,51275,51274,51237,50820,50821,50818,50802,50813,50882,50803)"

query_labs_le = f"""
SELECT 
    xx.icustay_id, 
    UNIX_SECONDS(TIMESTAMP(f.charttime)) as timestp, 
    f.itemid, 
    f.valuenum
FROM (
    SELECT subject_id, hadm_id, icustay_id, intime, outtime
    FROM `physionet-data.mimiciii_clinical.icustays`
    GROUP BY subject_id, hadm_id, icustay_id, intime, outtime
) as xx 
INNER JOIN `physionet-data.mimiciii_clinical.labevents` as f 
  ON f.hadm_id = xx.hadm_id 
  AND TIMESTAMP(f.charttime) >= TIMESTAMP_SUB(TIMESTAMP(xx.intime), INTERVAL 1 DAY) 
  AND TIMESTAMP(f.charttime) <= TIMESTAMP_ADD(TIMESTAMP(xx.outtime), INTERVAL 1 DAY)
WHERE f.itemid IN {lab_items_le} 
AND valuenum IS NOT NULL
ORDER BY f.hadm_id, timestp, f.itemid
"""

d = client.query(query_labs_le).to_dataframe()
d.to_csv(exportdir + 'labs_le.csv', index=False, sep='|')
print(f"Saved labs_le.csv ({len(d)} rows)")

--- Extracting Labs (Labevents) ---
Saved labs_le.csv (10612611 rows)


## Real-time UO

In [15]:
print("--- Extracting Urine Output ---")

uo_items = "(40055,43175,40069,40094,40715,40473,40085,40057,40056,40405,40428,40096,40651,226559,226560,227510,226561,227489,226584,226563,226564,226565,226557,226558)"

query_uo = f"""
SELECT 
    icustay_id, 
    UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
    itemid, 
    value
FROM `physionet-data.mimiciii_clinical.outputevents`
WHERE icustay_id IS NOT NULL 
AND value IS NOT NULL 
AND itemid IN {uo_items}
ORDER BY icustay_id, charttime, itemid
"""

d = client.query(query_uo).to_dataframe()
d.to_csv(exportdir + 'uo.csv', index=False, sep='|')
print(f"Saved uo.csv ({len(d)} rows)")

--- Extracting Urine Output ---
Saved uo.csv (3400332 rows)


## Pre-admission UO

In [16]:
print("--- Extracting Pre-admission UO ---")

query_preadm_uo = """
SELECT DISTINCT 
    oe.icustay_id, 
    UNIX_SECONDS(TIMESTAMP(oe.charttime)) as charttime, 
    oe.itemid, 
    oe.value,
    TIMESTAMP_DIFF(TIMESTAMP(ic.intime), TIMESTAMP(oe.charttime), MINUTE) as datediff_minutes
FROM `physionet-data.mimiciii_clinical.outputevents` oe, `physionet-data.mimiciii_clinical.icustays` ic
WHERE oe.icustay_id = ic.icustay_id 
AND itemid IN (40060, 226633)
ORDER BY icustay_id, charttime, itemid
"""

d = client.query(query_preadm_uo).to_dataframe()
d.to_csv(exportdir + 'preadm_uo.csv', index=False, sep='|')
print(f"Saved preadm_uo.csv ({len(d)} rows)")

--- Extracting Pre-admission UO ---
Saved preadm_uo.csv (12853 rows)


## Real-time input from metavision

- Records with no rate = STAT
- Records with rate = INFUSION
- fluids corrected for tonicity

In [17]:
print("--- Extracting Fluid Input (MV) ---")

mv_items = "(225158,225943,226089,225168,225828,225823,220862,220970,220864,225159,220995,225170,225825,227533,225161,227531,225171,225827,225941,225823,225825,225941,225825,228341,225827,30018,30021,30015,30296,30020,30066,30001,30030,30060,30005,30321,30006,30061,30009,30179,30190,30143,30160,30008,30168,30186,30211,30353,30159,30007,30185,30063,30094,30352,30014,30011,30210,46493,45399,46516,40850,30176,30161,30381,30315,42742,30180,46087,41491,30004,42698,42244)"

query_fluid_mv = f"""
WITH t1 AS (
    SELECT 
        icustay_id, 
        UNIX_SECONDS(TIMESTAMP(starttime)) as starttime, 
        UNIX_SECONDS(TIMESTAMP(endtime)) as endtime, 
        itemid, 
        amount, 
        rate,
        CASE 
            WHEN itemid IN (30176,30315) THEN amount * 0.25
            WHEN itemid IN (30161) THEN amount * 0.3
            WHEN itemid IN (30020,30015,225823,30321,30186,30211,30353,42742,42244,225159) THEN amount * 0.5
            WHEN itemid IN (227531) THEN amount * 2.75
            WHEN itemid IN (30143,225161) THEN amount * 3
            WHEN itemid IN (30009,220862) THEN amount * 5
            WHEN itemid IN (30030,220995,227533) THEN amount * 6.66
            WHEN itemid IN (228341) THEN amount * 8
            ELSE amount 
        END as tev
    FROM `physionet-data.mimiciii_clinical.inputevents_mv`
    WHERE icustay_id IS NOT NULL 
    AND amount IS NOT NULL 
    AND itemid IN {mv_items}
)
SELECT 
    icustay_id, 
    starttime, 
    endtime, 
    itemid, 
    ROUND(CAST(amount AS NUMERIC), 3) as amount, 
    ROUND(CAST(rate AS NUMERIC), 3) as rate, 
    ROUND(CAST(tev AS NUMERIC), 3) as tev
FROM t1
ORDER BY icustay_id, starttime, itemid
"""

d = client.query(query_fluid_mv).to_dataframe()
d.to_csv(exportdir + 'fluid_mv.csv', index=False, sep='|')
print(f"Saved fluid_mv.csv ({len(d)} rows)")

--- Extracting Fluid Input (MV) ---
Saved fluid_mv.csv (957959 rows)


## Real-time input from carevue

- In CAREVUE, all records are considered STAT doses!!
- fluids corrected for tonicity

In [18]:
print("--- Extracting Fluid Input (CV) ---")

query_fluid_cv = f"""
WITH t1 AS (
    SELECT 
        icustay_id, 
        UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
        itemid, 
        amount,
        CASE 
            WHEN itemid IN (30176,30315) THEN amount * 0.25
            WHEN itemid IN (30161) THEN amount * 0.3
            WHEN itemid IN (30020,30321, 30015,225823,30186,30211,30353,42742,42244,225159,225159,225159) THEN amount * 0.5
            WHEN itemid IN (227531) THEN amount * 2.75
            WHEN itemid IN (30143,225161) THEN amount * 3
            WHEN itemid IN (30009,220862) THEN amount * 5
            WHEN itemid IN (30030,220995,227533) THEN amount * 6.66
            WHEN itemid IN (228341) THEN amount * 8
            ELSE amount 
        END as tev
    FROM `physionet-data.mimiciii_clinical.inputevents_cv`
    WHERE amount IS NOT NULL 
    AND itemid IN {mv_items}
)
SELECT 
    icustay_id, 
    charttime, 
    itemid, 
    ROUND(CAST(amount AS NUMERIC), 3) as amount, 
    ROUND(CAST(tev AS NUMERIC), 3) as tev
FROM t1
ORDER BY icustay_id, charttime, itemid
"""

d = client.query(query_fluid_cv).to_dataframe()
d.to_csv(exportdir + 'fluid_cv.csv', index=False, sep='|')
print(f"Saved fluid_cv.csv ({len(d)} rows)")

--- Extracting Fluid Input (CV) ---
Saved fluid_cv.csv (3135918 rows)


## Pre-admission fluid intake

In [19]:
print("--- Extracting Pre-admission Fluid ---")

query_preadm_fluid = """
WITH mv AS (
    SELECT ie.icustay_id, SUM(ie.amount) as sum
    FROM `physionet-data.mimiciii_clinical.inputevents_mv` ie
    JOIN `physionet-data.mimiciii_clinical.d_items` ci ON ie.itemid = ci.itemid
    WHERE ie.itemid IN (30054,30055,30101,30102,30103,30104,30105,30108,226361,226363,226364,226365,226367,226368,226369,226370,226371,226372,226375,226376,227070,227071,227072)
    GROUP BY icustay_id
), cv AS (
    SELECT ie.icustay_id, SUM(ie.amount) as sum
    FROM `physionet-data.mimiciii_clinical.inputevents_cv` ie
    JOIN `physionet-data.mimiciii_clinical.d_items` ci ON ie.itemid = ci.itemid
    WHERE ie.itemid IN (30054,30055,30101,30102,30103,30104,30105,30108,226361,226363,226364,226365,226367,226368,226369,226370,226371,226372,226375,226376,227070,227071,227072)
    GROUP BY icustay_id
)
SELECT 
    pt.icustay_id,
    CASE 
        WHEN mv.sum IS NOT NULL THEN mv.sum
        WHEN cv.sum IS NOT NULL THEN cv.sum
        ELSE NULL 
    END as inputpreadm
FROM `physionet-data.mimiciii_clinical.icustays` pt
LEFT JOIN mv ON mv.icustay_id = pt.icustay_id
LEFT JOIN cv ON cv.icustay_id = pt.icustay_id
ORDER BY icustay_id
"""

d = client.query(query_preadm_fluid).to_dataframe()
d.to_csv(exportdir + 'preadm_fluid.csv', index=False, sep='|')
print(f"Saved preadm_fluid.csv ({len(d)} rows)")

--- Extracting Pre-admission Fluid ---
Saved preadm_fluid.csv (61532 rows)


## Vasopressors from metavision

- Drugs converted in noradrenaline-equivalent
- Body weight assumed 80 kg when missing

In [20]:
print("--- Extracting Vasopressors (MV) ---")

# Note: We check specifically for 'mcg/kg/min' and 'mcg/min' as they appear in BigQuery data
query_vaso_mv = """
SELECT 
    icustay_id, 
    itemid, 
    UNIX_SECONDS(TIMESTAMP(starttime)) as starttime, 
    UNIX_SECONDS(TIMESTAMP(endtime)) as endtime, 
    CASE 
        -- Norepinephrine (Norad)
        WHEN itemid IN (30120,221906,30047) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate AS NUMERIC), 3)
        WHEN itemid IN (30120,221906,30047) AND rateuom='mcg/min' THEN ROUND(CAST(rate/80 AS NUMERIC), 3)
        
        -- Epinephrine (Epi)
        WHEN itemid IN (30119,221289) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate AS NUMERIC), 3)
        WHEN itemid IN (30119,221289) AND rateuom='mcg/min' THEN ROUND(CAST(rate/80 AS NUMERIC), 3)
        
        -- Vasopressin
        WHEN itemid IN (30051,222315) AND rate > 0.2 THEN ROUND(CAST(rate*5/60 AS NUMERIC), 3) -- U/h
        WHEN itemid IN (30051,222315) AND rateuom='units/min' THEN ROUND(CAST(rate*5 AS NUMERIC), 3)
        WHEN itemid IN (30051,222315) AND rateuom='units/hour' THEN ROUND(CAST(rate*5/60 AS NUMERIC), 3)
        
        -- Phenylephrine
        WHEN itemid IN (30128,221749,30127) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate*0.45 AS NUMERIC), 3)
        WHEN itemid IN (30128,221749,30127) AND rateuom='mcg/min' THEN ROUND(CAST(rate*0.45 / 80 AS NUMERIC), 3)
        
        -- Dopamine
        WHEN itemid IN (221662,30043,30307) AND rateuom='mcg/kg/min' THEN ROUND(CAST(rate*0.01 AS NUMERIC), 3)
        WHEN itemid IN (221662,30043,30307) AND rateuom='mcg/min' THEN ROUND(CAST(rate*0.01/80 AS NUMERIC), 3)
        
        ELSE NULL 
    END as rate_std
FROM `physionet-data.mimiciii_clinical.inputevents_mv`
WHERE itemid IN (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307) 
AND rate IS NOT NULL 
AND statusdescription != 'Rewritten'
ORDER BY icustay_id, itemid, starttime
"""

d = client.query(query_vaso_mv).to_dataframe()
d.to_csv(exportdir + 'vaso_mv.csv', index=False, sep='|')
print(f"Saved vaso_mv.csv ({len(d)} rows)")

--- Extracting Vasopressors (MV) ---
Saved vaso_mv.csv (159318 rows)


## Vasopressors from carevue

- Same comments as above

In [21]:
print("--- Extracting Vasopressors (CV) ---")

query_vaso_cv = """
SELECT 
    icustay_id, 
    itemid, 
    UNIX_SECONDS(TIMESTAMP(charttime)) as charttime, 
    CASE 
        -- Norepinephrine
        WHEN itemid IN (30120,221906,30047) AND rateuom IN ('mcgkgmin', 'mcg/kg/min') THEN ROUND(CAST(rate AS NUMERIC), 3)
        WHEN itemid IN (30120,221906,30047) AND rateuom IN ('mcgmin', 'mcg/min') THEN ROUND(CAST(rate/80 AS NUMERIC), 3)
        
        -- Epinephrine
        WHEN itemid IN (30119,221289) AND rateuom IN ('mcgkgmin', 'mcg/kg/min') THEN ROUND(CAST(rate AS NUMERIC), 3)
        WHEN itemid IN (30119,221289) AND rateuom IN ('mcgmin', 'mcg/min') THEN ROUND(CAST(rate/80 AS NUMERIC), 3)
        
        -- Vasopressin
        WHEN itemid IN (30051,222315) AND rate > 0.2 THEN ROUND(CAST(rate*5/60 AS NUMERIC), 3)
        WHEN itemid IN (30051,222315) AND rateuom IN ('Umin', 'units/min') AND rate < 0.2 THEN ROUND(CAST(rate*5 AS NUMERIC), 3)
        WHEN itemid IN (30051,222315) AND rateuom IN ('Uhr', 'units/hour') THEN ROUND(CAST(rate*5/60 AS NUMERIC), 3)
        
        -- Phenylephrine
        WHEN itemid IN (30128,221749,30127) AND rateuom IN ('mcgkgmin', 'mcg/kg/min') THEN ROUND(CAST(rate*0.45 AS NUMERIC), 3)
        WHEN itemid IN (30128,221749,30127) AND rateuom IN ('mcgmin', 'mcg/min') THEN ROUND(CAST(rate*0.45 / 80 AS NUMERIC), 3)
        
        -- Dopamine
        WHEN itemid IN (221662,30043,30307) AND rateuom IN ('mcgkgmin', 'mcg/kg/min') THEN ROUND(CAST(rate*0.01 AS NUMERIC), 3)
        WHEN itemid IN (221662,30043,30307) AND rateuom IN ('mcgmin', 'mcg/min') THEN ROUND(CAST(rate*0.01/80 AS NUMERIC), 3)
        
        ELSE NULL 
    END as rate_std
FROM `physionet-data.mimiciii_clinical.inputevents_cv`
WHERE itemid IN (30128,30120,30051,221749,221906,30119,30047,30127,221289,222315,221662,30043,30307) 
AND rate IS NOT NULL
ORDER BY icustay_id, itemid, charttime
"""

d = client.query(query_vaso_cv).to_dataframe()
d.to_csv(exportdir + 'vaso_cv.csv', index=False, sep='|')
print(f"Saved vaso_cv.csv ({len(d)} rows)")

--- Extracting Vasopressors (CV) ---
Saved vaso_cv.csv (923686 rows)


## Mechanical ventilation

In [22]:
print("--- Extracting Mechanical Ventilation ---")

query_mechvent = """
SELECT
    icustay_id, 
    UNIX_SECONDS(TIMESTAMP(charttime)) as charttime,
    MAX(
      CASE
        WHEN itemid IS NULL OR value IS NULL THEN 0 
        WHEN itemid = 720 AND value != 'Other/Remarks' THEN 1  -- VentTypeRecorded
        WHEN itemid = 467 AND value = 'Ventilator' THEN 1 -- O2 delivery device
        WHEN itemid IN (
          445, 448, 449, 450, 1340, 1486, 1600, 224687, -- minute volume
          639, 654, 681, 682, 683, 684, 224685, 224684, 224686, -- tidal volume
          218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 224747, -- High/Low/Peak/Mean/Neg insp force
          221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 224750, 227187, -- Insp pressure
          543, -- PlateauPressure
          5865, 5866, 224707, 224709, 224705, 224706, -- APRV pressure
          60, 437, 505, 506, 686, 220339, 224700, -- PEEP
          3459, -- high pressure relief
          501, 502, 503, 224702, -- PCV
          223, 667, 668, 669, 670, 671, 672, -- TCPCV
          157, 158, 1852, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 8382, 227809, 227810, -- ETT
          224701 -- PSVlevel
        ) THEN 1
        ELSE 0
      END
    ) as MechVent,
    MAX(
        CASE 
          WHEN itemid IS NULL OR value IS NULL THEN 0
          WHEN itemid = 640 AND value = 'Extubated' THEN 1
          WHEN itemid = 640 AND value = 'Self Extubation' THEN 1
          ELSE 0
        END
    ) as Extubated,
    MAX(
        CASE 
          WHEN itemid IS NULL OR value IS NULL THEN 0
          WHEN itemid = 640 AND value = 'Self Extubation' THEN 1
          ELSE 0
        END
    ) as SelfExtubated
FROM `physionet-data.mimiciii_clinical.chartevents`
WHERE value IS NOT NULL
AND itemid IN (
    640, 720, 467, 
    445, 448, 449, 450, 1340, 1486, 1600, 224687, 
    639, 654, 681, 682, 683, 684, 224685, 224684, 224686, 
    218, 436, 535, 444, 459, 224697, 224695, 224696, 224746, 224747, 
    221, 1, 1211, 1655, 2000, 226873, 224738, 224419, 224750, 227187, 
    543, 
    5865, 5866, 224707, 224709, 224705, 224706, 
    60, 437, 505, 506, 686, 220339, 224700, 
    3459, 
    501, 502, 503, 224702, 
    223, 667, 668, 669, 670, 671, 672, 
    157, 158, 1852, 3398, 3399, 3400, 3401, 3402, 3403, 3404, 8382, 227809, 227810, 
    224701
)
GROUP BY icustay_id, charttime
ORDER BY icustay_id, charttime
"""

d = client.query(query_mechvent).to_dataframe()
d.to_csv(exportdir + 'mechvent.csv', index=False, sep='|')
print(f"Saved mechvent.csv ({len(d)} rows)")

--- Extracting Mechanical Ventilation ---
Saved mechvent.csv (2217611 rows)
